In [154]:
from itertools import zip_longest
import datetime as dt
import requests
import json

from typing import Optional, List, Dict, Iterable, Tuple
from pandas.core.frame import DataFrame

import numpy as np
import praw
from psaw import PushshiftAPI
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

sns.set()

TODO list:

1. Get all flaired submissions in a week
2. Check for duplicates (once)
3. Parse the response, get id, use PRAW to get post info
4. Filter by flair
5. Run over various weeks, combine

In [2]:
def chunker(seq: Iterable, size: int) -> Iterable:
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [3]:
# reddit = praw.Reddit(client_id='TiogCJcLNTKaXQ', 
#                      client_secret='vtroPTNpBpqh2Qze5umXTlpmh7A', 
#                      username='Moral_Judgement_Bot', 
#                      password='A not very secure password.',
#                      user_agent='AITA bot test script')

# api = PushshiftAPI(r=reddit)

In [99]:
type(df)

pandas.core.frame.DataFrame

In [120]:
def get_posts_between(start: int, end: int, field_names: Optional[List[str]] = None) -> pd.DataFrame:
    if field_names is None:
        field_names = ['id', 'title', 'selftext', 'score', 'author']

    api_url = 'https://api.pushshift.io/reddit/submission/search'

    req = requests.get(api_url, 
            params = {
                "subreddit": "AmITheAsshole",
                "sort": "desc",
                "sort_type": "score",
                "after": start_epoch,
                "before": end_epoch,
                "limit": 10000
            })
    
    res = json.loads(req.content)
    
    filtered = filter(
            lambda x: all(name in x.keys() for name in field_names),
            res['data']
            )

    data = list(map(
                lambda x: tuple(x[field] for field in field_names), 
                filtered
                ))
    
    df = pd.DataFrame(data, columns=field_names)
    df['name'] = 't3_' + df['id']
    
    return df

In [121]:
def request_info(names: List[str], useragent: str = 'MoralJudgementIncoming') -> List[Dict]:
    all_names = ','.join(names)
    
    res = requests.get('https://www.reddit.com/api/info.json',
            params={'id': all_names},
            headers={'User-agent': useragent})
    
    posts_info = json.loads(res.content)['data']['children']
    
#     print(len(posts_info), len(names))
    assert len(posts_info) == len(names), ValueError("Didn't get all posts' info")
    
    return posts_info

In [122]:
def get_details(df: pd.DataFrame) -> pd.DataFrame:
    
    # TODO: add error handling if a chunk fails
    name_chunks = chunker(df.name, 100)

    all_info = []

    for chunk in tqdm(name_chunks):
        info = request_info(chunk)

        all_info.extend(info)


    field_names = ['id', 'title', 'selftext', 'score', 'edited', 'link_flair_text']

    data = list(map(
                lambda x: (x['data'][field] for field in field_names), 
                all_info
                ))

    df_new = pd.DataFrame(data, columns=field_names)
    df_new = df_new.fillna('')
    df_new = df_new.set_index('id')
    
    return df_new

In [123]:
start_epoch = int(dt.datetime(2019, 7, 30, 0).timestamp())
end_epoch = int(dt.datetime(2019, 7, 30, 12).timestamp())

df_basic = get_posts_between(start_epoch, end_epoch)

In [88]:
df = get_details(df_basic)

7it [00:07,  1.06s/it]


In [129]:
dt.datetime(2019, 7, 30).timestamp()

1564437600.0

In [131]:
foo = [1, 2, 3, 4, 5]

In [135]:
a, b, c, *_ = foo

In [155]:
def iterate_days(start: Tuple[int], end: Tuple[int], diff: int = 24) -> Tuple[dt.datetime, dt.datetime]:
    """
    
    Args:
        start: (year, month, day), start date
        end: (year, month, day), end date
        diff: number of hours to iterate over
    """ # TODO: somehow make it possible to change timedelta maybe
    start_dt = dt.datetime(*start)
    end_dt = dt.datetime(*end)
    
    assert start_dt < end_dt, ValueError("Start date should be before end date")
    
    current_dt = dt.datetime(*start)
    
    while current_dt != end_dt:
        start = current_dt
        end = current_dt + dt.timedelta(hours=diff)
        
        current_dt = end
        yield (start, end)

In [164]:
days = iterate_days((2019, 1, 1), (2019, 1, 5), 12)
for s,e in days:
    print(s, e)

2019-01-01 00:00:00 2019-01-01 12:00:00
2019-01-01 12:00:00 2019-01-02 00:00:00
2019-01-02 00:00:00 2019-01-02 12:00:00
2019-01-02 12:00:00 2019-01-03 00:00:00
2019-01-03 00:00:00 2019-01-03 12:00:00
2019-01-03 12:00:00 2019-01-04 00:00:00
2019-01-04 00:00:00 2019-01-04 12:00:00
2019-01-04 12:00:00 2019-01-05 00:00:00


In [151]:
dt.datetime(2019, 5, 5) > dt.datetime(2019, 1, 1)

True

In [105]:
df.index = df.id

In [110]:
df.set_index('id')

,title,selftext,score,edited,link_flair_text
id,,,,,
cjpipi,AITA for correcting my cousin publically over ...,The title probably gives 90% of the informatio...,13112,1.56452e+09,Not the A-hole
cjk291,AITA for not cutting up a red onion for my mot...,[deleted],12324,1.56445e+09,
cjmxsu,AITA for technically stealing my neighbor's cat?,"OK, I'l try to shorten the story so it's diges...",4015,False,
cjpfqn,AITA for refusing a “child free” couple on a f...,My family runs a tour company that operates sm...,2935,1.5645e+09,Everyone Sucks
cjn60g,AITA for being rude to my girlfriend's friends?,My girlfriend invited over a long term friend ...,2539,False,Not the A-hole
cjihbf,WIBTA for telling my sister that her husband s...,"Throwaway acct but this is real, and a difficu...",2194,1.56454e+09,Not the A-hole
cjli39,AITA for thinking my dad is weird and perverted?,[deleted],1439,1.56452e+09,Not the A-hole
cjkcka,AITA for telling my Dad that just because he p...,So my dad lost his job and so we put the house...,1465,False,Not the A-hole
cjjh45,Accept Your Judgement: A Deep Dive,Rule 3: Accept your judgement. Perhaps our mos...,1986,False,META
